In [1]:
import torch 
from torch import nn
from d2l import torch as d2l

def corr2d(X,K):
    """
    计算二位互相关运算
    """
    h,w=K.shape
    Y=torch.zeros((X.shape[0]-h+1,X.shape[1]-w+1))
    for i in range(Y.shape[0]):
        for j in range (Y.shape[1]):
            Y[i,j]=(X[i:i+h,j:j+w]*K).sum()
    return Y

In [3]:
X=torch.tensor([[0,1,2],[3,4,5],[6,7,8]])
K=torch.tensor([[0,1],[2,3]])
corr2d(X,K)

tensor([[19., 25.],
        [37., 43.]])

## 实现二维卷积层

In [4]:
class Conv2D(nn.Module):
    def __init__(self,kernel_size):
        super().__init__()
        self.weight=nn.Parameter(torch.rand(kernel_size))
        self.bias=nn.Parameter(torch.zeors(1))
        
    def forward(self,x):
        return corr2d(x,self.weight)+self.bias

In [15]:
X=torch.ones(6,8)
X[:,2:6]=0
X
# 模拟边缘（黑-白-黑）

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [16]:
K=torch.tensor([[1,-1]])
Y=corr2d(X,K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [17]:
conv2d=nn.Conv2d(1,1,kernel_size=(1,2),bias=False)

X=X.reshape((1,1,6,8))
Y=Y.reshape((1,1,6,7))

for i in range(10):
    Y_hat=conv2d(X)
    l=(Y_hat-Y)**2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:]-=3e-2*conv2d.weight.grad
    if(i+1)%2==0:
        print(f'batch{i+1},loss{l.sum():.3f}')

batch2,loss10.999
batch4,loss3.501
batch6,loss1.266
batch8,loss0.490
batch10,loss0.196


In [18]:
conv2d.weight.data

tensor([[[[ 0.9441, -1.0347]]]])